In [1]:
import torch
import torch.nn as nn

### RoPE 的数学基础

RoPE 的核心思想是将位置信息通过旋转矩阵嵌入到 Transformer 的注意力机制中。具体来说，它对查询向量（query）和键向量（key）应用旋转操作，使得位置信息以相对距离的形式自然融入注意力计算。

#### 1. 基本旋转矩阵
对于一个二维向量 $\mathbf{x} = [x_0, x_1]$ 和位置 $m$，RoPE 使用旋转矩阵对其进行变换：
$$
R(m, \theta) = \begin{pmatrix}
\cos(m\theta) & -\sin(m\theta) \\
\sin(m\theta) & \cos(m\theta)
\end{pmatrix}
$$
其中：
- $m$ 是 token 在序列中的位置（例如 0, 1, 2, ...）。
- $\theta$ 是旋转角度，通常与嵌入维度的索引相关。

应用到向量上：
$$
\mathbf{x}' = R(m, \theta) \mathbf{x} = \begin{pmatrix}
\cos(m\theta) & -\sin(m\theta) \\
\sin(m\theta) & \cos(m\theta)
\end{pmatrix} \begin{pmatrix}
x_0 \\
x_1
\end{pmatrix} = \begin{pmatrix}
x_0 \cos(m\theta) - x_1 \sin(m\theta) \\
x_0 \sin(m\theta) + x_1 \cos(m\theta)
\end{pmatrix}
$$

#### 2. 多维嵌入的扩展
对于更高维度的嵌入向量（例如维度 $d$），RoPE 将向量分成 $d/2$ 个二维子向量，每对子向量使用不同的旋转角度 $\theta_i$：
- 假设嵌入向量为 $\mathbf{x} = [x_0, x_1, x_2, x_3, ..., x_{d-2}, x_{d-1}]$。
- 对于第 $i$ 对子向量 $[x_{2i}, x_{2i+1}]$，应用旋转矩阵：
$$
R(m, \theta_i) = \begin{pmatrix}
\cos(m\theta_i) & -\sin(m\theta_i) \\
\sin(m\theta_i) & \cos(m\theta_i)
\end{pmatrix}
$$
其中 $\theta_i$ 是第 $i$ 对子向量对应的旋转频率。

完整的变换可以写为：
$$
\mathbf{x}'_m = R_d(m, \Theta) \mathbf{x} = [x_0 \cos(m\theta_0) - x_1 \sin(m\theta_0), x_0 \sin(m\theta_0) + x_1 \cos(m\theta_0), ..., x_{d-2} \cos(m\theta_{d/2-1}) - x_{d-1} \sin(m\theta_{d/2-1}), x_{d-2} \sin(m\theta_{d/2-1}) + x_{d-1} \cos(m\theta_{d/2-1})]
$$
其中 $\Theta = [\theta_0, \theta_1, ..., \theta_{d/2-1}]$ 是频率向量。

#### 3. 频率 $\theta_i$ 的定义
在标准 RoPE 中，旋转角度 $\theta_i$ 由以下公式定义：
$$
\theta_i = \frac{1}{\text{base}^{2i/d}}
$$
其中：
- $\text{base}$ 是一个超参数，通常取值为 $10000$（与原始 Transformer 的位置编码一致）。
- $d$ 是嵌入维度。
- $i$ 是维度索引，范围为 $0, 1, 2, ..., d/2-1$。

对应的逆频率（即代码中的 `inv_freq`）为：

$$
\text{inv\_freq}_i = \frac{1}{\text{base}^{2i/d}}
$$

在实践中，$\theta_i$ 是一个递减序列，低维度的频率变化慢，高维度的频率变化快，这种设计有助于捕捉不同尺度的位置关系。

#### 4. 注意力中的应用
在 Transformer 的自注意力机制中，RoPE 被应用到查询向量 $\mathbf{q}$ 和键向量 $\mathbf{k}$ 上：
- $\mathbf{q}_m = R_d(m, \Theta) \mathbf{q}$
- $\mathbf{k}_n = R_d(n, \Theta) \mathbf{k}$

注意力分数变为：
$$
\text{Attention}(\mathbf{q}_m, \mathbf{k}_n) = (\mathbf{q}_m)^T \mathbf{k}_n = (R_d(m, \Theta) \mathbf{q})^T (R_d(n, \Theta) \mathbf{k})
$$

由于旋转矩阵的性质，这可以简化为只依赖于相对位置 $m - n$ 的形式（具体推导涉及三角恒等式，这里略去细节），从而实现相对位置编码。

---

### NTK 缩放的调整

NTK 缩放的目标是让 RoPE 能够适应比训练时更长的序列长度。它的核心思想是通过调整 $\text{base}$ 值，拉伸频率分布，使得旋转角度对更长的序列仍然有效。

#### 1. 原始 $\text{base}$ 的局限
在标准 RoPE 中，$\text{base}$ 是固定的（例如 10000）。当序列长度超过训练时的最大长度（即 `max_position_embeddings`）时，旋转角度 $\theta_i \cdot m$ 可能变得过大，导致位置编码失去区分度（因为 $\cos$ 和 $\sin$ 函数的周期性使得角度过大时难以区分）。

#### 2. NTK 的 $\text{base}$ 调整公式
NTK 通过动态调整 $\text{base}$ 来解决这个问题。代码中的调整公式为：
$$
\text{base}' = \text{base} \cdot \left( \left( \text{factor} \cdot \frac{\text{seq\_len}}{\text{max\_position\_embeddings}} \right) - (\text{factor} - 1) \right)^{\frac{d}{d-2}}
$$

- $\text{base}$：原始基础频率（例如 10000）。
- $\text{factor}$：缩放因子，通常大于 1（如 2.0），控制扩展的强度。
- $\text{seq\_len}$：当前序列长度。
- $\text{max\_position\_embeddings}$：训练时的最大位置嵌入数。
- $d$：嵌入维度。

调整后的逆频率变为：
$$
\text{inv\_freq}_i = \frac{1}{(\text{base}')^{2i/d}}
$$

#### 3. 调整的效果
- 当 $\text{seq\_len} > \text{max\_position\_embeddings}$ 时，$\frac{\text{seq\_len}}{\text{max\_position\_embeddings}} > 1$，使得 $\text{base}' > \text{base}$。
- $\text{base}'$ 变大，$\theta_i = \frac{1}{(\text{base}')^{2i/d}}$ 变小，旋转角度 $m \cdot \theta_i$ 的增长速度变慢。
- 结果：即使 $m$ 变得很大（对应长序列中的位置），旋转角度仍然保持在合理范围内，避免过快的周期性重复。

指数项 $\frac{d}{d-2}$ 是一个平滑因子，确保频率调整在高维度下不会过于剧烈。

#### 4. 数学示例
假设：
- $\text{base} = 10000$，$d = 128$，$\text{max\_position\_embeddings} = 2048$，$\text{seq\_len} = 4096$，$\text{factor} = 2.0$。

计算：
$$
\frac{\text{seq\_len}}{\text{max\_position\_embeddings}} = \frac{4096}{2048} = 2
$$
$$
\text{factor} \cdot 2 - (\text{factor} - 1) = 2 \cdot 2 - (2 - 1) = 4 - 1 = 3
$$
$$
\frac{d}{d-2} = \frac{128}{126} \approx 1.0159
$$
$$
\text{base}' = 10000 \cdot 3^{1.0159} \approx 10000 \cdot 3.047 \approx 30470
$$

调整后的 $\theta_i$：
$$
\theta_i = \frac{1}{30470^{2i/128}}
$$

相比原来的 $\theta_i = \frac{1}{10000^{2i/128}}$，频率变小，旋转角度随 $m$ 增长更慢，适合更长的序列。

---

### 总结公式

1. **标准 RoPE 频率**：
   $$
   \theta_i = \frac{1}{\text{base}^{2i/d}}
   $$

2. **NTK 调整后的 $\text{base}$**：
   $$
   \text{base}' = \text{base} \cdot \left( \left( \text{factor} \cdot \frac{\text{seq\_len}}{\text{max\_position\_embeddings}} \right) - (\text{factor} - 1) \right)^{\frac{d}{d-2}}
   $$

3. **NTK 调整后的频率**：
   $$
   \theta_i = \frac{1}{(\text{base}')^{2i/d}}
   $$

4. **旋转矩阵**（不变）：
   $$
   R(m, \theta_i) = \begin{pmatrix}
   \cos(m\theta_i) & -\sin(m\theta_i) \\
   \sin(m\theta_i) & \cos(m\theta_i)
   \end{pmatrix}
   $$

In [2]:
def init_rope_default(dim, base=10000):
    inv_freq = 1.0 / base ** (torch.arange(0, dim, 2, dtype=torch.int64).float() / dim)
    attention_factor = 1.0
    return inv_freq, attention_factor

In [3]:
def init_rope_ntk(dim, base=10000, factor=2.0, seq_len=4096, max_position_embeddings=2048):
    # 为了简单，这里假设了seq_len是4096，但是具体训练的时候max_position_embeddings是2048
    seq_len = seq_len if seq_len is not None and seq_len > max_position_embeddings else max_position_embeddings
    base = base * ((factor * seq_len / max_position_embeddings) - (factor - 1)) ** (dim / dim - 2)
    inv_freq = 1.0 / base ** (torch.arange(0, dim, 2, dtype=torch.int64).float() / dim)
    attention_factor = 1.0
    return inv_freq, attention_factor


### 线性插值
#### 数学公式

- **标准 RoPE 频率**：
  $$
  \theta_i = \frac{1}{\text{base}^{2i/d}}
  $$
- **线性缩放后的频率**：
  $$
  \theta_i' = \frac{1}{(\text{base} \cdot \text{factor})^{2i/d}} = \frac{\theta_i}{\text{factor}}
  $$
- **旋转角度**：
  对于位置 $m$ ，旋转角度变为：
  $$
  m \cdot \theta_i' = m \cdot \frac{\theta_i}{\text{factor}}
  $$
  相当于将位置索引 $m$ 压缩了 $\text{factor}$ 倍。

#### 原理

- **目的**：通过减小频率 $\theta_i$，使得旋转角度 $m \cdot \theta_i$ 的增长变慢，从而在更长的序列中保持位置编码的区分度。
- **等价性**：对 `inv_freq` 除以 `factor` 等价于将位置索引 $m$ 缩放到 $m / \text{factor}$，从而模拟较短的序列。
- **优点**：简单直接，计算开销低。
- **局限**：线性缩放假设频率调整是均匀的，可能无法很好地适应极长序列或复杂模式。

#### 示例

假设 $\text{base} = 10000$，$d = 128$，$\text{factor} = 2$：

- 原始 $\theta_0 = \frac{1}{10000^{0/128}} = 1$，$\theta_1 = \frac{1}{10000^{2/128}} \approx 0.630957$。
- 调整后 $\theta_0' = \frac{1}{2} = 0.5$，$\theta_1' = \frac{0.630957}{2} \approx 0.315478$。
- 对于 $m = 4096$，原始角度 $4096 \cdot 0.630957 \approx 2584$（弧度），调整后 $4096 \cdot 0.315478 \approx 1292$，变化更平缓。

---

### YaRN（Yarn-augmented Rotary Embeddings）

#### 数学公式

- **外推频率**：
  $$
  \text{inv\_freq\_extrapolation}_i = \frac{1}{\text{base}^{2i/d}}
  $$
- **内插频率**：
  $$
  \text{inv\_freq\_interpolation}_i = \frac{1}{(\text{factor} \cdot \text{base}^{2i/d})}
  $$
- **维度校正范围**：
  $$
  \text{low} = \lfloor \frac{d \cdot \log(\text{max\_position\_embeddings} / (\beta_{\text{fast}} \cdot 2\pi))}{2 \cdot \log(\text{base})} \rfloor, \quad \text{high} = \lceil \frac{d \cdot \log(\text{max\_position\_embeddings} / (\beta_{\text{slow}} \cdot 2\pi))}{2 \cdot \log(\text{base})} \rceil
  $$
- **线性插值因子**：
  $$
  \text{ramp}_i = \text{clamp}\left( \frac{i - \text{low}}{\text{high} - \text{low}}, 0, 1 \right)
  $$
- **最终频率**：
  $$
  \text{inv\_freq}_i = \text{inv\_freq\_interpolation}_i \cdot (1 - \text{ramp}_i) + \text{inv\_freq\_extrapolation}_i \cdot \text{ramp}_i
  $$

#### 原理

- **内插与外推结合**：YaRN 在低维度使用内插频率（类似 Linear Scaling，适应长序列），在高维度使用外推频率（保留原始 RoPE 的分辨率）。
- **动态调整**：通过 `beta_fast` 和 `beta_slow`，控制哪些维度需要调整，线性插值因子 `ramp` 实现平滑过渡。
- **注意力因子**：`attention_factor` 可用于调整注意力分数，增强外推效果。
- **优点**：比 Linear Scaling 更灵活，能更好地平衡短距离和高频细节的捕捉。
- **局限**：实现较复杂，依赖超参数调优。

#### 示例

假设 $\text{base} = 10000$，$d = 128$，$\text{factor} = 2$，$\text{max\_position\_embeddings} = 2048$，$\beta_{\text{fast}} = 32$，$\beta_{\text{slow}} = 1$：

- $\text{inv\_freq\_extrapolation}_0 = 1$，$\text{inv\_freq\_interpolation}_0 = 0.5$。
- 计算 `low ≈ 18`，`high ≈ 43`（近似值）。
- 对于 $i = 0$（维度 0），$\text{ramp}_0 = 0$，$\text{inv\_freq}_0 = 0.5$。
- 对于 $i = 32$（维度 64），$\text{ramp}_{32} \approx 0.56$，$\text{inv\_freq}_{32}$ 是内插和外推的混合。

---

### 对比与总结

| 方法  | 调整方式 | 数学公式 | 优点  | 局限  |
| --- | --- | --- | --- | --- |
| **Linear** | 均匀缩放频率 | $\theta_i' = \theta_i / \text{factor}$ | 简单高效 | 缺乏灵活性 |
| **YaRN** | 内插与外推的动态混合 | 加权平均 $\text{inv\_freq}_i$ | 更细粒度的频率控制 | 实现复杂，参数敏感 |
| **NTK** | 调整 $\text{base}$ | $\text{base}' = f(\text{seq\_len})$ | 理论支持强，扩展性好 | 对超长序列效果有限 |

- **Linear Scaling**：直接缩放频率，适用于中等长度的扩展。
- **YaRN**：通过维度相关的插值，兼顾短距离和高频细节，适合更复杂场景。
- **NTK**（前述讨论）：动态调整 $\text{base}$，强调理论上的平滑扩展。
